In [70]:
import numpy as np
import duckdb
import pandas as pd

In [71]:
def df_to_DBtable(DB,df,table): # repeated
    """
    Creates a persistent table in DuckDB from the DataFrame content.
    """
    con = duckdb.connect(DB)
    con.register(table, df)
    con.execute(f'CREATE TABLE {table} AS SELECT * FROM {table}')
    con.close()

## Join two sources into one table

In [72]:
con = duckdb.connect('integration.duckdb')
nationalites = con.execute(f'SELECT * FROM nationalitiesClean_Madrid').df()
households = con.execute(f'SELECT * FROM householdClean_Madrid').df()
con.close()

# display(nationalites)
# display(households)


# # checking if all section codes are same in both tables

# difference = [item for item in sectionsH if item not in sectionsN]
# len(difference)

# print(difference)
# print(len(difference))

result = households.merge(nationalites, how='inner', on=["Madrid_section","Year"])
# result = pd.merge(households,nationalites, how='inner')


df_to_DBtable('integration.duckdb',result, 'integratedTable')


In [73]:
# check if all tables in integration.duckdb are created
con = duckdb.connect('integration.duckdb')
int_tables = con.execute(f'SELECT table_name FROM information_schema.tables').df()
con.close()
int_tables

,table_name
0,nationalitiesClean_Madrid
1,householdClean_Madrid
2,integratedTable
